In [ ]:
import os
from dotenv import load_dotenv
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
load_dotenv()

# Setup Selenium WebDriver
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.91 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    # Add more user agents as needed
]
user_agent = random.choice(user_agents)

chrome_options = Options()
chrome_options.add_argument(f"user-agent={user_agent}")
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

In [3]:
def random_sleep(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

In [4]:
# LinkedIn Login URL
driver.get('https://www.linkedin.com/login')

# Login using environment variables
username = driver.find_element(By.ID, 'username')
password = driver.find_element(By.ID, 'password')

linkedin_username = os.getenv('LINKEDIN_USERNAME')
linkedin_password = os.getenv('LINKEDIN_PASSWORD')

username.send_keys(linkedin_username)
password.send_keys(linkedin_password)

# Click login button
driver.find_element(By.XPATH, "//button[@type='submit']").click()

time.sleep(8)

# Wait for login to complete
try:
    WebDriverWait(driver, 10).until(EC.url_contains('feed'))
except:
    print("Login failed or took too long.")
    driver.quit()  # Exit if login fails

# Load or create the Excel file to store data
excel_file = 'information.xlsx'

if os.path.exists(excel_file):
    df_existing = pd.read_excel(excel_file)
else:
    df_existing = pd.DataFrame(columns=['Name', 'Bio', 'Company', 'Phone Number', 'Email', 'Profile Link'])

def initial_navigation():
    """Perform some initial navigation before going to the connections page."""
    driver.get('https://www.linkedin.com/feed/')
    random_sleep(4, 9)  # Random delay to mimic human behavior

initial_navigation()

In [5]:
# def search_linkedin(queries):
#     """Search for specific profiles on LinkedIn using Selenium and Beautiful Soup."""
#     print("Starting LinkedIn search with Beautiful Soup...")
       
#     # List to collect all profile links
#     all_profile_links = []

#     # Iterate over each query in the queries list
#     for query in queries:
#         print(f"Searching for query: {query}")
        
#         # Format the search URL
#         search_url = f"https://www.linkedin.com/search/results/people/?keywords={query.replace(' ', '%20')}"
#         driver.get(search_url)
        
#         while True:
#             try:
#                 # Wait for content to load
#                 WebDriverWait(driver, 10).until(
#                     EC.presence_of_element_located((By.CLASS_NAME, 'search-results-container'))
#                 )
                
#                 # Scroll down to the bottom of the page to load all profiles
#                 last_height = driver.execute_script("return document.body.scrollHeight")
#                 while True:
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     time.sleep(2)  # Wait for new profiles to load

#                     # Check if we've reached the bottom
#                     new_height = driver.execute_script("return document.body.scrollHeight")
#                     if new_height == last_height:
#                         break
#                     last_height = new_height

#                 # Get the page source and parse with Beautiful Soup
#                 page_source = driver.page_source
#                 soup = BeautifulSoup(page_source, 'html.parser')
#                 profiles = soup.find_all('a', class_='app-aware-link')

#                 # Collect profile links from the current page
#                 for profile in profiles:
#                     link = profile.get('href')
#                     if link and "linkedin.com/in/" in link and link not in all_profile_links:
#                         all_profile_links.append(link)

#                 print(f"Collected {len(all_profile_links)} profile links so far for '{query}'.")

#                 # Locate and click the "Suivant" button to go to the next page
#                 try:
#                     next_button = WebDriverWait(driver, 10).until(
#                         EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Suivant']"))
#                     )
#                     driver.execute_script("arguments[0].click();", next_button)
#                     time.sleep(5)  # Wait for the next page to load
#                 except Exception:
#                     # If the "Suivant" button is not found or clickable, break the loop
#                     print(f"No more pages to load for '{query}'.")
#                     break

#             except Exception as e:
#                 print(f"Error occurred for '{query}':", e)
#                 break

#         print(f"Completed search for '{query}' with a total of {len(all_profile_links)} profiles.")

#     profile_links_array = np.array(all_profile_links)
#     print(f"Total profiles found across all queries: {len(profile_links_array)}")
    
#     return profile_links_array


In [6]:
def search_linkedin(queries, max_pages=5):
    """Search for specific profiles on LinkedIn using Selenium and Beautiful Soup."""
    print("Starting LinkedIn search with Beautiful Soup...")
    
    # List to collect all profile links
    all_profile_links = []

    # Iterate over each query in the queries list
    for query in queries:
        print(f"Searching for query: {query}")
        
        # Format the search URL
        search_url = f"https://www.linkedin.com/search/results/people/?keywords={query.replace(' ', '%20')}"
        driver.get(search_url)
        
        page_count = 0  # Initialize page counter for each query

        while page_count < max_pages:
            try:
                # Wait for content to load
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'search-results-container'))
                )
                
                # Scroll down to the bottom of the page to load all profiles
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)  # Wait for new profiles to load

                    # Check if we've reached the bottom
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height

                # Get the page source and parse with Beautiful Soup
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                profiles = soup.find_all('a', class_='app-aware-link')

                # Collect profile links from the current page
                for profile in profiles:
                    link = profile.get('href')
                    if link and "linkedin.com/in/" in link and link not in all_profile_links:
                        all_profile_links.append(link)

                print(f"Collected {len(all_profile_links)} profile links so far for '{query}'.")

                # Increment page counter
                page_count += 1

                # Locate and click the "Suivant" button to go to the next page
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Suivant']"))
                )
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(5)  # Wait for the next page to load

            except Exception as e:
                print(f"No more pages to load for '{query}' or 'Suivant' button not clickable:", e)
                break

        print(f"Completed search for '{query}' with a total of {len(all_profile_links)} profiles.")

    profile_links_array = np.array(all_profile_links)
    print(f"Total profiles found across all queries: {len(profile_links_array)}")
    
    return profile_links_array


In [7]:
queries = [
    "lauréat ENSAM Rabat",
    "alumni ENSAM Rabat",
    "diplômé ENSAM Rabat",
    # "ancien étudiant ENSAM Rabat",
    # "réseau ENSAM Rabat",
    # "ingénieur mécanique ENSAM Rabat",
    # "ingénieur électromécanique ENSAM Rabat",
    # "ingénieur génie industriel ENSAM Rabat",
    # "spécialiste maintenance industrielle ENSAM Rabat",
    # "ingénieur production ENSAM Rabat",
    # "expert en logistique ENSAM Rabat",
    # "consultant ENSAM Rabat",
    # "stagiaire ENSAM Rabat",
    # "jeune diplômé ENSAM Rabat",
    # "professionnel expérimenté ENSAM Rabat",
    # "manager ENSAM Rabat",
    # "directeur de projet ENSAM Rabat",
    # "réseau professionnel ENSAM Rabat",
    # "communauté ENSAM Maroc",
    # "groupe ENSAM Maroc",
    # "gestion de projet ENSAM Rabat",
    # "développement durable ENSAM Rabat",
    # "énergie renouvelable ENSAM Rabat",
    # "automatisation industrielle ENSAM Rabat",
    # "maintenance et sécurité ENSAM Rabat"
]

In [8]:
profile_links_array = search_linkedin(queries, max_pages=2)
# profile_links_array = search_linkedin(queries)

Starting LinkedIn search with Beautiful Soup...
Searching for query: lauréat ENSAM Rabat
Collected 25 profile links so far for 'lauréat ENSAM Rabat'.
Collected 46 profile links so far for 'lauréat ENSAM Rabat'.
Completed search for 'lauréat ENSAM Rabat' with a total of 46 profiles.
Searching for query: alumni ENSAM Rabat
Collected 61 profile links so far for 'alumni ENSAM Rabat'.
Collected 64 profile links so far for 'alumni ENSAM Rabat'.
No more pages to load for 'alumni ENSAM Rabat' or 'Suivant' button not clickable: Message: 

Completed search for 'alumni ENSAM Rabat' with a total of 64 profiles.
Searching for query: diplômé ENSAM Rabat
Collected 80 profile links so far for 'diplômé ENSAM Rabat'.
Collected 95 profile links so far for 'diplômé ENSAM Rabat'.
Completed search for 'diplômé ENSAM Rabat' with a total of 95 profiles.
Total profiles found across all queries: 95


In [9]:
# profile_links_array = profile_links_array[1:10]

In [10]:
def extract_name(soup):
    """Extracts the full name from the profile page."""
    try:
        name_element = soup.find('h1', {'class': 'text-heading-xlarge inline t-24 v-align-middle break-words'})
        return name_element.text.strip() if name_element else "Not available"
    except Exception as e:
        print(f"Error extracting name: {e}")
        return "Not available"

def extract_bio(soup):
    """Extracts the bio from the profile page."""
    try:
        bio_element = soup.find('div', {'class': 'text-body-medium break-words'})
        return bio_element.text.strip() if bio_element else "Not available"
    except Exception as e:
        print(f"Error extracting bio: {e}")
        return "Not available"


In [11]:
def extract_education_details():
    """Navigates to the education details section by modifying the current URL and extracts diploma, location, and graduation date."""
    try:
        # Get the current URL of the page
        current_url = driver.current_url

        # Navigate to the education details page
        driver.get(f"{current_url}details/education/")
        time.sleep(2)  # Wait for the education details page to load

        # Parse the education details page with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Locate diploma, location, and graduation date elements using BeautifulSoup
        diploma_element = soup.select_one("a.optional-action-target-wrapper.display-flex.flex-column.full-width span.t-14.t-normal span:nth-of-type(1)")
        location_element = soup.select_one("a.optional-action-target-wrapper.display-flex.flex-column.full-width > div.display-flex.flex-wrap.align-items-center.full-height > div.display-flex > div.display-flex.full-width > div.display-flex.align-items-center.mr1.hoverable-link-text.t-bold > span:nth-of-type(1)")
        graduation_date_element = soup.select_one("a.optional-action-target-wrapper.display-flex.flex-column.full-width > span.t-14.t-normal.t-black--light > span.pvs-entity__caption-wrapper")

        # Extract the data
        diploma = diploma_element.text.strip() if diploma_element else "Not available"
        location = location_element.text.strip() if location_element else "Not available"
        graduation_date = graduation_date_element.text.strip() if graduation_date_element else "Not available"
        
        return diploma, location, graduation_date
    except Exception as e:
        print(f"Error extracting education details: {e}")
        return "Not available", "Not available", "Not available"


In [12]:
def extract_profiles_data(profile_links):
    """Extracts data from multiple LinkedIn profile links, including name, bio, and education details."""
    all_profile_data = []

    for profile_url in profile_links:
        print(f"Extracting data for profile: {profile_url}")

        # Visit the main profile page and parse with BeautifulSoup for name and bio
        driver.get(profile_url)
        time.sleep(2)  # Wait for the main profile page to load
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        profile_data = {}
        profile_data["Nom"] = extract_name(soup)
        profile_data["Bio"] = extract_bio(soup)

        # Extract education details by navigating directly within the helper function
        profile_data["Diplôme"], profile_data["Endroit"], profile_data["Quand"] = extract_education_details()

        all_profile_data.append(profile_data)
        print(f"Data extracted for profile: {profile_url}")

    return all_profile_data


In [13]:
# Extract data from the profiles
all_profiles_data = extract_profiles_data(profile_links_array)

# Print the extracted data
for profile in all_profiles_data:
    print(profile)


Extracting data for profile: https://www.linkedin.com/in/marwane-hachimi?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADPSZ3ABKPEXrkQ1VMlajnQaaTqBL8ZGmXQ
Data extracted for profile: https://www.linkedin.com/in/marwane-hachimi?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADPSZ3ABKPEXrkQ1VMlajnQaaTqBL8ZGmXQ
Extracting data for profile: https://www.linkedin.com/in/ACoAACbfWM8BvrvUEyOyek0jyHc0YfbwRovYBaA?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACbfWM8BvrvUEyOyek0jyHc0YfbwRovYBaA
Data extracted for profile: https://www.linkedin.com/in/ACoAACbfWM8BvrvUEyOyek0jyHc0YfbwRovYBaA?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACbfWM8BvrvUEyOyek0jyHc0YfbwRovYBaA
Extracting data for profile: https://www.linkedin.com/in/ACoAAC3pxyIBK8bvW5izn_d1NHh84mvMvQ7KtQo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC3pxyIBK8bvW5izn_d1NHh84mvMvQ7KtQo
Data extracted for profile: https://www.linkedin.com/in/ACoAAC3pxyIBK8bvW5izn_d1NHh84mvMvQ7KtQo?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC3pxyI

In [14]:
# Convert the extracted data to a DataFrame
df = pd.DataFrame(all_profiles_data)

# Print the DataFrame
df


,Nom,Bio,Diplôme,Endroit,Quand
0,Marwane HACHIMI,Ingénieur d'état en Qualité et Environnement,"Diplôme d'ingénieur, Génie des Matériaux, Qual...",ENSAM Rabat,sept. 2021 - juil. 2024
1,Abdelhamid Laaraj,Developer/SAP Technical Consultant @Beyond Tec...,"Engineering, Data Science & AI",ENSAM Rabat,oct. 2021 - juil. 2024
2,Zakariae Elyafi,Customer Service Engineer (CSE) at Siemens Hea...,"Engineer's degree, Biomedical Engineering",ENSAM Rabat,sept. 2021 - juil. 2024
3,Abdelilah Ahhttouche,Consultant en Systèmes Embarqués,"Ingénieur d'Etat, Génie Electromécanique et Sy...",École Nationale Supérieure d'Arts et Métiers (...,sept. 2016 - juil. 2021
4,Ziad Akallal,Étudiant ingénieur en génie mécanique et énerg...,juin 2019 - juin 2025,École Nationale Supérieure d'Arts et Métiers (...,juin 2019 - juin 2025
...,...,...,...,...,...
90,Maryam ABIA,biomedical engineering student,Master's Degree in Biomedical Instrumentation ...,Faculty of Medicine and Pharmacy of Marrakech,oct. 2023 - juil. 2025
91,Wijdane BENABID,Étudiante à ENSAM Rabat,"Diplôme d'ingénieur, Génie industriel et techn...",ENSAM Rabat,sept. 2022 - juin 2025
92,Said ARNOUZ,Élève ingénieur en ingénierie des systèmes num...,"Diplôme d'ingénieur , Energie électrique et in...",ENSAM Rabat,sept. 2022 - 2025
93,Oussama EL YOUSOUFI,Étudiant en génie Mécanique à ENSAM Rabat,"Diplôme d'ingénieur, Ingénierie mécanique",ENSAM Rabat,2022 - 2025


In [15]:
# Save the DataFrame to a CSV file
df.to_csv('profiles_data.csv', index=False)


In [2]:
!pip freeze > requirements.txt